In [1]:
!unzip fraudcalls.zip

Archive:  fraudcalls.zip
  inflating: fraud_call.file         


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import math

def loadData(file):
    text, labels = [], []
    with open(file, 'r') as data: 
        for line in data:
            split = line.strip().split('\t')
            if (split[0] != ""):
                labels.append(split[0])
                text.append(split[1])

    return text, labels

alltext, alllabels = loadData("fraud_call.file")

norm = 0
fraud  = 0

for i in range(0,len(alllabels)):
    if (alllabels[i] == "fraud"):
        fraud+=1
    if (alllabels[i] == "normal"):
        norm+=1

print(len(alllabels))
print(norm)
print(fraud)

print(math.floor(norm*.65))
print(math.floor(fraud*.65))
print(math.floor(norm*.2))
print(math.floor(fraud*.2))
print(math.floor(norm*.15))
print(math.floor(fraud*.15))

l = np.array(alllabels)
t = np.array(alltext)

wholedataset = np.column_stack((l,t))
np.random.shuffle(wholedataset)

#gonna turn these into numpy arrays
train = []  
verify = []
test = []

i=0 
while (len(train) < 3437):
    if (wholedataset[i][0] == "normal"):
        train.append(wholedataset[i])
    i+=1
print(i)
while (len(verify) < 1057):
    if (wholedataset[i][0] == "normal"):
        verify.append(wholedataset[i])
    i+=1
while (len(test) < 795):
    if (wholedataset[i][0] == "normal"):
        test.append(wholedataset[i])
    i+=1
i=0
while (len(train) < 3437 + 414):
    if (wholedataset[i][0] == "fraud"):
        train.append(wholedataset[i])
    i+=1
while (len(verify) < 1057+127):
    if (wholedataset[i][0] == "fraud"):
        verify.append(wholedataset[i])
    i+=1
while (len(test) < 795+97):
    if (wholedataset[i][0] == "fraud"):
        test.append(wholedataset[i])
    i+=1

nptrain = np.stack(train, axis=0)
npverify = np.stack(verify, axis=0)
nptest = np.stack(test, axis=0)

np.random.shuffle(nptrain)
np.random.shuffle(npverify)
np.random.shuffle(nptest)

print(nptrain[:5])
print(npverify[:5])
print(nptest[:5]) 

5927
5289
638
3437
414
1057
127
793
95
3856
[['normal' 'Sure. I hope your complaints will have gone till then.']
 ['normal'
  '"Life is nothing wen v get everything". But "life is everything wen v miss something ". Real value of people wil be realized only in their absence.... gud mrng']
 ['normal'
  "Oh, the grand is having a bit of a party but it doesn't mention any cover charge so it's probably first come first served"]
 ['normal' 'Cool. I am  &lt;#&gt;  inches long. hope you like them big!']
 ['normal' 'I am on the way to ur home']]
[['normal'
  "cThen i thk shd b enuff.. Still got conclusion n contents pg n references.. I'll b doing da contents pg n cover pg.."]
 ['normal' 'What class of  &lt;#&gt;  reunion?']
 ['normal'
  'I have 2 sleeping bags, 1 blanket and paper and  phone details. Anything else?']
 ['normal' 'Y lei?']
 ['normal' 'will you like to be spoiled? :)']]
[['normal' 'Dunno why you ask me.']
 ['normal'
  'I have been so busy with my new job that I have not had the ti

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

vectorizer = CountVectorizer()
traindata = vectorizer.fit_transform(nptrain[:,1])
valdata = vectorizer.transform(npverify[:,1])
testdata = vectorizer.transform(nptest[:,1])


In [4]:
classifier = LogisticRegression(penalty='l2', fit_intercept=True, C=1)

classifier.fit(traindata, nptrain[:,0])
print("training accuracy = ", classifier.score(traindata, nptrain[:,0]))
print("validation accuracy = ", classifier.score(valdata, npverify[:,0]))

classifier.predict(vectorizer.transform(["this is a phone call where i commit insurance fraud"]))

training accuracy =  0.9968839262529213
validation accuracy =  0.9763513513513513


array(['normal'], dtype='<U910')

In [5]:
import pickle 

modelname = "logregmodel.sav"
pickle.dump(classifier, open(modelname, "wb"))

In [6]:
#need the vectorizer too!

pickle.dump(vectorizer, open("vectorizer.sav", "wb"))

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

numfraud = 0
for i in range(0,len(npverify)):
    if npverify[i][0] == "fraud":
        numfraud +=1

print(numfraud)

valpreds = classifier.predict(valdata)
confusion = confusion_matrix(npverify[:,0],valpreds)

#19 false negatives
print(confusion)
precision = confusion[0][0] / (confusion[0][0] + confusion[1][0])
recall = confusion[0][0] / (confusion[0][0] + confusion[0][1])
f1s =  2 * ((recall * precision)/(recall + precision))
print(precision)
print(recall)
print(f1s)

127
[[ 102   25]
 [   3 1054]]
0.9714285714285714
0.8031496062992126
0.8793103448275863
